### In this notebook, we run simulations for producing output files for Gephi to do graph analysis. The color codes were build with SVD.

In [1]:
import pandas as pd
import numpy as np
import random
from random import shuffle, randrange
from collections import defaultdict
import utilities

import matplotlib.pyplot as plt
import seaborn as sns

import json

import AgentClass
import const
import networkx as nx
from config import num_agents,number_of_bits
from scipy import stats
from collections import defaultdict
import copy

from sklearn.decomposition import TruncatedSVD

In [2]:
%matplotlib inline

In [3]:
random.seed(1)
np.random.seed(1)

In [4]:
def get_tau_distr():
        lower = 0
        upper = 1
        mu = 0.5
        sigma = 0.1
        N = 1000
        
        samples = stats.truncnorm.rvs(
          (lower-mu)/sigma,(upper-mu)/sigma,loc=mu,scale=sigma,size=N)
        
        return samples

create environment with small world network graph

In [5]:
def create_environment():
    list_agents = []
    tau_distr = get_tau_distr()
    
    for i in range(num_agents):
        in_state = [first_bit.pop(), second_bit.pop(), third_bit.pop(), fourth_bit.pop(), fifth_bit.pop()
                   , sixth_bit.pop(), seventh_bit.pop(), eigth_bit.pop(), ninth_bit.pop(), tenth_bit.pop()]
        a = AgentClass.Agent(name='agent{}'.format(i), tau=random.choice(tau_distr), initial_state=in_state)
        list_agents.append(a)
        
    # create network
    G = nx.newman_watts_strogatz_graph(num_agents, 10, 0.5, seed= 0)
#    nx.draw(G, with_labels=True, font_weight='bold') # to draw agents
    df = nx.to_pandas_adjacency(G, dtype=int)
    
    tmp_edges = df.apply(lambda row: row.to_numpy().nonzero()).to_dict()
    edges = {k: v[0].tolist() for k, v in tmp_edges.items()}
    
    # make random connections with agents
    for k, v in edges.items():
        for ngh in v:
            list_agents[k].add_neighbors(list_agents[ngh])
            
    return list_agents

In [6]:
def get_network_df(list_agents):
    network_df = pd.DataFrame({'Agent Name':[], 'Neighbors':[]})
    for agt in list_agents:
        neighbors = agt.get_neighbors_name()
        network_df = network_df.append({'Agent Name':agt.name, 
                                        'Neighbors':neighbors}, ignore_index=True)
    return network_df

Get the coherence matrix values

In [7]:
random.seed(1)
  
constants = const.Constants()
coherence_matrix = constants.get_coh_matrix().tolist()


# Color codes

Initialize the initial knowledge state for 300 agents with half having 1's and other with 0's for each knowledge bit

In [8]:
sim_list_agents = []
tau_distr = get_tau_distr()

first_sim_bit = [0 for i in range(300//2)] + [1 for i in range(300//2)]
second_sim_bit = [0 for i in range(300//2)] + [1 for i in range(300//2)]
third_sim_bit = [0 for i in range(300//2)] + [1 for i in range(300//2)]
fourth_sim_bit = [0 for i in range(300//2)] + [1 for i in range(300//2)]
fifth_sim_bit = [0 for i in range(300//2)] + [1 for i in range(300//2)]
sixth_sim_bit = [0 for i in range(300//2)] + [1 for i in range(300//2)]
seventh_sim_bit = [0 for i in range(300//2)] + [1 for i in range(300//2)]
eigth_sim_bit = [0 for i in range(300//2)] + [1 for i in range(300//2)]
ninth_sim_bit = [0 for i in range(300//2)] + [1 for i in range(300//2)]
tenth_sim_bit = [0 for i in range(300//2)] + [1 for i in range(300//2)]

shuffle(first_sim_bit)
shuffle(second_sim_bit)
shuffle(third_sim_bit)
shuffle(fourth_sim_bit)
shuffle(fifth_sim_bit)
shuffle(sixth_sim_bit)
shuffle(seventh_sim_bit)
shuffle(eigth_sim_bit)
shuffle(ninth_sim_bit)
shuffle(tenth_sim_bit)

for i in range(300):
    in_state = [first_sim_bit.pop(), second_sim_bit.pop(), third_sim_bit.pop(), fourth_sim_bit.pop(), fifth_sim_bit.pop()
                   , sixth_sim_bit.pop(), seventh_sim_bit.pop(), eigth_sim_bit.pop(), ninth_sim_bit.pop(), tenth_sim_bit.pop()]
    a = AgentClass.Agent(name='agent{}'.format(i), tau=random.choice(tau_distr), initial_state=in_state)
    sim_list_agents.append(a)
        
# create network
G = nx.newman_watts_strogatz_graph(300, 10, 0.5, seed= 0)
#    nx.draw(G, with_labels=True, font_weight='bold') # to draw agents
df = nx.to_pandas_adjacency(G, dtype=int)
    
tmp_edges = df.apply(lambda row: row.to_numpy().nonzero()).to_dict()
edges = {k: v[0].tolist() for k, v in tmp_edges.items()}
    
# make random connections with agents
for k, v in edges.items():
    for ngh in v:
        sim_list_agents[k].add_neighbors(sim_list_agents[ngh])
            

In [9]:
def run_color_simulation(alpha, coh_matrix, list_agents, end_time):
    
    d = []
    for t in range(end_time): 
        # compute next state for all agents
        for agt in list_agents:
            agt.update_knowledge(alpha, coh_matrix) 
         
        # keep record of current record and all other values
        for agt in list_agents:
            row = {'Agent_Name':agt.name,
                   'Agent_Dissonance':agt.dissonance_lst,
                   'Time':t,
                   'Current State':agt.knowledge_state,
                   'Next State':agt.next_state}
            
            d.append(row)
        
        agent_states = {agent.name: utilities.bool2int(agent.knowledge_state) for agent in list_agents}
       
            
        # now update all agents next state with computed next state
        for agt in list_agents:
            agt.knowledge_state = agt.next_state
            agt.next_state = None
            agt.dissonance_lst = None
            
    return pd.DataFrame(d)

In [10]:
record__color_df = run_color_simulation(0, coherence_matrix, copy.deepcopy(sim_list_agents), 10000)
record__color_df['Current_Knowledge_State'] =  record__color_df['Current State']
record__color_df['Current State'] = record__color_df['Current State'].apply(lambda row: utilities.bool2int(row))
record__color_df['Next State'] = record__color_df['Next State'].apply(lambda row: utilities.bool2int(row))
record__color_df['alpha'] = str(0)
color_df1 = record__color_df.groupby(['Current State', 'Next State']).size().to_frame('Count').reset_index()

In [11]:
len(record__color_df)

3000000

Use SVD on state transition matrix to get the color codes

In [12]:
color_df = pd.crosstab(record__color_df['Current State'], record__color_df['Agent_Name'])
# get color codes
# states_bin_df = pd.DataFrame.from_dict({v:[int(ch) for ch in '{0:010b}'.format(v)] for v in range(1024)}).T
svd = TruncatedSVD(n_components=3, n_iter=7, random_state=42)
t = svd.fit_transform(color_df)  
i = np.interp(t, (t.min(), t.max()), (0, 255))
i = i.astype(int)
color_codes = {index: {'r':row[0], 'g':row[1], 'b':row[2], 'a': 0} for index, row in enumerate(i)}

The script above ran the simulations only to get the color codes, while we run again for the simulations of 100 agents

# Run Simulations

The graphs for Gephi were produced when timestamp in [0,24, 49, 74, 99]

In [13]:
def run_simulation(alpha, coh_matrix, list_agents, end_time):
    
    d = []
    selected_time_stamps = [0,24, 49, 74, 99]
    for t in range(end_time):    
        # compute next state for all agents
        for agt in list_agents:
            agt.update_knowledge(alpha, coh_matrix) 
         
        # keep record of current record and all other values
        for agt in list_agents:
            row = {'Agent_Name':agt.name,
                   'Agent_Dissonance':agt.dissonance_lst,
                   'Time':t,
                   'Current State':agt.knowledge_state,
                   'Next State':agt.next_state}
            
            d.append(row)
        
        agent_states = {agent.name: utilities.bool2int(agent.knowledge_state) for agent in list_agents}
        if t in selected_time_stamps:
            G=nx.Graph()

            for agent in list_agents:
                agent_num = int(''.join([ch for ch in agent.name if ch.isdigit()]))
                agent_neighbors = [(int(''.join([ch for ch in neighbor_name if ch.isdigit()])), agent_states[neighbor_name]) for neighbor_name in agent.neighbors]
                nghbrs = [int(''.join([ch for ch in neighbor_name if ch.isdigit()])) for neighbor_name in agent.neighbors]
                agent_state = utilities.bool2int(agent.knowledge_state)
                G.add_node(agent_num)
                G.node[agent_num]['viz'] = {'color': color_codes[agent_state]}
                G.node[agent_num]['state'] = agent_state

                for n_agent in nghbrs:
                    G.add_edge(agent_num, n_agent)

            nx.write_gexf(G, "poster_outputs/gephi/network_alpha_{}_t_{}.gexf".format(alpha,t))
            
        # now update all agents next state with computed next state
        for agt in list_agents:
            agt.knowledge_state = agt.next_state
            agt.next_state = None
            agt.dissonance_lst = None
            
    return pd.DataFrame(d)

In [14]:
first_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
second_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
third_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
fourth_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
fifth_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
sixth_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
seventh_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
eigth_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
ninth_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]
tenth_bit = [0 for i in range(num_agents//2)] + [1 for i in range(num_agents//2)]

shuffle(first_bit)
shuffle(second_bit)
shuffle(third_bit)
shuffle(fourth_bit)
shuffle(fifth_bit)
shuffle(sixth_bit)
shuffle(seventh_bit)
shuffle(eigth_bit)
shuffle(ninth_bit)
shuffle(tenth_bit)

In [15]:
end_simulation_time = 100
alphas = [0,.25,.5,.75,1.0]
    
# first create environment
agents_list = create_environment()
    
# get network of the agents
agent_network_df = get_network_df(agents_list) 
    
results = {}
    
# for saving
agent_network_df.to_json('simulations_new/test_network.json',orient='records', lines=True)
        
results['seed'] = 1
results['coherence_matrix'] = coherence_matrix
        
    
results['alphas'] = defaultdict(list)
all_df = pd.DataFrame()

In [16]:
kbits_dict = defaultdict(list)

for i in range(1024):
    kbits_dict['State'].append(i)
    kbits_dict['binary'].append('{0:010b}'.format(i))
    
    kbits_dict['binary_list'].append(list('{0:010b}'.format(i)))

kbits_mapper_df = pd.DataFrame(kbits_dict)
kbits_mapper_df.set_index('State', inplace=True)
kbits_mapper_df.drop('binary', inplace=True, axis=1)
kbits_mapper_df_dict = kbits_mapper_df.to_dict(orient='index')

In [17]:
# run simulation
for alpha in alphas:
    record_df = None
    record_df = run_simulation(alpha, coherence_matrix, copy.deepcopy(agents_list), end_simulation_time)
    record_df['Current_Knowledge_State'] =  record_df['Current State']
    record_df['Current State'] = record_df['Current State'].apply(lambda row: utilities.bool2int(row))
    record_df['Next State'] = record_df['Next State'].apply(lambda row: utilities.bool2int(row))
    record_df['alpha'] = str(alpha)
    all_df = all_df.append(record_df)
    df = record_df.groupby(['Current State', 'Next State']).size().to_frame('Count').reset_index()

In [18]:
# t_df= all_df[all_df['alpha']=='0']
# cross_df = pd.crosstab(t_df['Current State'], t_df['Next State'])

After running simulations assign color codes to each node of __Knowledge__

In [19]:
import networkx as nx

In [20]:
for a in alphas:
    G = nx.DiGraph()
    x = all_df[all_df['alpha']==str(a)].groupby(['Current State', 'Next State']).size().to_frame('Count')
    x.reset_index(inplace=True)
    for _, row in x.iterrows():
        curr_state = row['Current State']

        G.add_node(curr_state)
        G.node[curr_state]['viz'] = {'color': color_codes[curr_state]}
        G.add_edge(curr_state, row['Next State'], weight=row['Count'])

    nx.write_gexf(G, "poster_outputs/gephi/knowledge_{}.gexf".format(a))

In [21]:
pd.DataFrame(coherence_matrix).to_csv('coh_matrix.csv')